In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [50]:
from collections import deque

operations={
    'XOR': '^',
    'OR': '|',
    'AND': '&',
}

def initialize_state(line,wires):
    w,val = line.strip().split(':')
    wires[w]=int(val)

def compute_wire(line,wires, missed):
    w1,op,w2, _, w3 = line.strip().split()
    try:
        wires[w3] = eval(f'{wires[w1]}{operations[op]}{wires[w2]}')
    except KeyError:
        missed.append(line)

def compute_number(wires):
    zkeys = [w for w in wires if  w.startswith('z')]
    bin_num_array = [ str(wires[f'z{i:02}']) for i in range(len(zkeys)-1,-1,-1)]
    return int(''.join(bin_num_array), 2)
    
def compute_number_string(wires,l='z'):
    zkeys = [w for w in wires if  w.startswith(l)]
    bin_num_array = [ str(wires[f'{l}{i:02}']) for i in range(len(zkeys)-1,-1,-1)]
    return int(''.join(bin_num_array), 2)
    

def part_one(path='input_data/test_24.txt'):
    wires={}
    missed_computes = deque([])    

    init=True
    with open(path,'r') as f:
        for line in f.readlines():
            if len(line.strip()) < 1:
                init=False
                continue
            if init:
                initialize_state(line, wires)
            else:
                compute_wire(line, wires, missed_computes)            
    while len(missed_computes) >0: 
        line = missed_computes.popleft()
        compute_wire(line, wires, missed_computes)

    number = compute_number_string(wires)
    print(compute_number_string(wires,l='x'),compute_number_string(wires,l='y'))
    return number

In [51]:
%%time
part_one()

13 31
CPU times: user 2.02 ms, sys: 1.65 ms, total: 3.67 ms
Wall time: 2.76 ms


2024

In [52]:
%%time
part_one('input_data/day_24.txt')

26495111766585 18168650014383
CPU times: user 12.8 ms, sys: 2.08 ms, total: 14.8 ms
Wall time: 13.7 ms


45213383376616

# Part Two

In [45]:
'ntg XOR fgs -> mjb'.split()

['ntg XOR fgs ', ' mjb']

[('ntg', 'XOR', 'fgs', 'mjb')]

In [202]:
from dataclasses import dataclass
import re
import numpy as np
from copy import deepcopy

@dataclass
class Wire:
    inputs: list 
    operation: str
    name: str
    def __str__(self):
        return f'{self.name}: {self.inputs[0]} {self.operation} {self.inputs[1]}'
    
operations={
    'XOR': '^',
    'OR': '|',
    'AND': '&',
}

class WireNetwork:
    
    def _initialize_state(self,line,wires):
        w,val = line.strip().split(':')
        wires[w]=int(val)
        
    def _get_compute_wire(self, line, calc_compute):
        in1, op, in2, w = re.findall(r'(\w+) (\w+) (\w+) -> (\w+)',line)[0]
        calc_compute[w]=Wire([in1,in2], op, w)

    def calculate_wire(self, wire, new_set=None):
        if wire in self.set_wires:
            return self.set_wires[wire]
            
        compute = self.wire_computes[wire] if new_set==None else new_set[wire]
        in1=self.calculate_wire(compute.inputs[0],new_set)
        in2=self.calculate_wire(compute.inputs[1],new_set)
        return eval(f'{in1}{operations[compute.operation]}{in2}')

    def wire_inputs(self, wire, dependency):
        if wire in self.set_wires:
            dependency.append(wire)
            return 
        compute = self.wire_computes[wire]
        dependency.append(wire)
        for i in range(2):
            self.wire_inputs(compute.inputs[i], dependency)
            
    def get_unique_inputs(self,wnum):
        contributing=[] 
        remove=[]
        self.wire_inputs(f'z{wnum:02}',contributing)
        self.wire_inputs(f'z{wnum-1:02}',remove)
        unique_set= list(set(contributing)-set(remove))
        return unique_set

    def swap_wires(self,w1,w2):
        tmp = self.wire_computes[w1]
        self.wire_computes[w1] = self.wire_computes[w2]
        self.wire_computes[w2] = tmp

            
    def calculate_bad_bits(self):
        carry=0 
        nbits = len([ w for w in self.set_wires.keys() if  w.startswith('x')])
        bad_bits=[-2]
        for b in range(nbits):
            x = self.calculate_wire(f'x{b:02}')
            y = self.calculate_wire(f'y{b:02}')
            z = self.calculate_wire(f'z{b:02}')
            bit_sum = x+y+carry
            if z != (bit_sum)%2:
                bad_bits.append(b)
                
            carry=int(bit_sum>1)
            
        bad_bits = np.array(bad_bits)
        print(bad_bits) 
        diffs_mask = (bad_bits[1:] - bad_bits[:-1]) > 1
        
        bad_bits = bad_bits[1:][diffs_mask]
        
        self.bad_bits = [ f'z{z:02}' for z in bad_bits]
        
    def __init__(self, data_path):
        wires={}
        computes = {}
        init=True
        
        with open(data_path,'r') as f:
            for line in f.readlines():
                if len(line.strip()) < 1:
                    init=False
                    continue
                if init:
                    self._initialize_state(line, wires)
                else:
                    self._get_compute_wire(line,computes)
                    
        self.set_wires = wires
        self.wire_computes =computes
        
        


In [221]:
test= WireNetwork('input_data/day_24.txt')

In [222]:
test.calculate_bad_bits()

[-2  9 10 11 12 13 14 27 28 39 40]


In [223]:
test.swap_wires('cnk','qwf')
test.swap_wires('z14','vhm')
test.swap_wires('mps','z27')
test.swap_wires('z39','msq')

In [226]:
','.join(sorted(['cnk','qwf',
'z14','vhm',
'mps','z27',
'z39','msq',]))

'cnk,mps,msq,qwf,vhm,z14,z27,z39'

In [224]:
test.calculate_bad_bits()

[-2]


In [200]:
for s in test.get_unique_inputs(4):
    if s in test.wire_computes:
        print(str(test.wire_computes[s]))

tjk: y04 XOR x04
z04: cpn XOR tjk
qjg: y03 AND x03
vph: vmk AND tbk
cpn: qjg OR vph


In [196]:
str(test.wire_computes['vmk'])
str(test.wire_computes['tbk'])

'vmk: dng OR wrc'

'tbk: x03 XOR y03'

In [188]:
for s in test.get_unique_inputs(5):
    if s in test.wire_computes:
        print(str(test.wire_computes[s]))

tvf: tjk AND cpn
qmn: x04 AND y04
z05: tfr XOR trt
tfr: tvf OR qmn
trt: x05 XOR y05


In [219]:
for s in test.get_unique_inputs(39):
    if s in test.wire_computes:
        print(str(test.wire_computes[s]))

z39: trn AND gpm
trn: y39 XOR x39
dsj: sfj AND wph
gpm: pqr OR dsj
pqr: x38 AND y38


In [213]:
for s in test.get_unique_inputs(28):
    if s in test.wire_computes:
        print(str(test.wire_computes[s]))

z28: bhb XOR mps
bhb: x28 XOR y28
mps: kqw XOR kqj


In [220]:
for w,v in test.wire_computes.items():
    if 'trn' in v.inputs:
        print(str(v))

msq: gpm XOR trn
z39: trn AND gpm


In [207]:
for s in test.get_unique_inputs(15):
    if s in test.wire_computes:
        print(str(test.wire_computes[s]))

tjk: y04 XOR x04
tvf: tjk AND cpn
ndq: gfn OR jmh
wwp: pjh OR gww
krg: brk OR fjj
hkm: x06 XOR y06
mjm: cnq AND dht
wmw: x02 XOR y02
trt: x05 XOR y05
pfd: x11 AND y11
jdr: bpv OR fsv
ptm: mjm OR gdd
qmn: x04 AND y04
gfj: mpd OR pfd
vvc: ncs OR frj
pjh: y01 AND x01
tbk: x03 XOR y03
gdd: x10 AND y10
dng: wwp AND wmw
dmq: x13 XOR y13
cnq: jsd OR qwf
hhp: gbf OR gnt
cpn: qjg OR vph
vmk: dng OR wrc
brk: y07 AND x07
dht: x10 XOR y10
jrh: y08 XOR x08
gnt: jrh AND krg
frj: y05 AND x05
fgv: rkm AND ndq
fgw: x00 AND y00
wgp: vvc AND hkm
z15: bbw XOR jnw
vhm: ndq XOR rkm
bpv: gfj AND gmr
ncs: tfr AND trt
bbw: vhm OR fgv
gfn: jdr AND dmq
bgb: x01 XOR y01
qwf: x09 XOR y09
mpd: fwf AND ptm
jsd: cnk AND hhp
jnw: y15 XOR x15
pqj: y06 AND x06
cnk: x09 AND y09
gmr: y12 XOR x12
fwf: y11 XOR x11
qjg: y03 AND x03
gbf: y08 AND x08
rkm: x14 XOR y14
wrc: x02 AND y02
gmp: pqj OR wgp
bjv: x07 XOR y07
fjj: bjv AND gmp
tfr: tvf OR qmn
gww: fgw AND bgb
fsv: y12 AND x12
vph: vmk AND tbk
jmh: y13 AND x13
